#### Imports & Setup

In [112]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
import faiss
import shelve
import pickle

import numpy                 as np
import matplotlib.pyplot     as plt
import matplotlib.image      as mpimg


from keras.preprocessing     import image
from deepface.detectors      import FaceDetector
from deepface.DeepFace       import build_model as build_verifier
from deepface.commons        import functions, distance as dst
from api_classes             import Representation
from uuid                    import uuid4
from tqdm                    import tqdm
from timeit                  import default_timer as timer

In [94]:
# Setting up paths
API_DIR      = os.path.join(os.path.dirname(os.path.realpath("__file__")))
DST_ROOT_DIR = os.path.join(API_DIR     , 'data')
RAW_DIR      = os.path.join(DST_ROOT_DIR, 'raw')
#GALLERY_DIR  = os.path.join(DST_ROOT_DIR, 'gallery')
TARGETS_DIR  = os.path.join(DST_ROOT_DIR, 'targets')
RDB_DIR      = os.path.join(DST_ROOT_DIR, 'database')
SVD_MDL_DIR  = os.path.join(API_DIR     , 'saved_models')
SVD_VRF_DIR  = os.path.join(SVD_MDL_DIR , 'verifiers')

directory_list       = [API_DIR, DST_ROOT_DIR, RAW_DIR, TARGETS_DIR, RDB_DIR,
                        SVD_MDL_DIR, SVD_VRF_DIR]
directory_list_names = ['api root', 'dataset root', 'raw', 'targets',
                        'rep. database', 's. models', 's. verifiers']

# Print path names
print('  -> Path names:')
for name, fp in zip(directory_list_names, directory_list):
    print(f'Directory {name}'.ljust(25), f': {fp}', sep='')
print('')

  -> Path names:
Directory api root       : /home/rpessoa/projects/loki/api
Directory dataset root   : /home/rpessoa/projects/loki/api/data
Directory raw            : /home/rpessoa/projects/loki/api/data/raw
Directory targets        : /home/rpessoa/projects/loki/api/data/targets
Directory rep. database  : /home/rpessoa/projects/loki/api/data/database
Directory s. models      : /home/rpessoa/projects/loki/api/saved_models
Directory s. verifiers   : /home/rpessoa/projects/loki/api/saved_models/verifiers



In [224]:
# NOTE: These functions were copied directly to the notebook to avoid conflicts
# and issues when importing from separate Python files. FastAPI seems to work
# with relative imports while IPYNP seems to work with absolute imports, so to
# avoid this issue alltogether, just keep the functions local to this demo.

# ______________________________________________________________________________
#                                   FUNCTIONS
# ------------------------------------------------------------------------------

def get_image_paths(root_path, file_types=('.jpg', '.png'), do_sort=True):
    """
    Gets the full paths of all 'file_types' files in the 'root_path' directory
    and its subdirectories. If the 'root_path' provided points to a file, this
    functions simply returns that path as a list with 1 element. The image path
    list is sorted if 'do_sort' flag is set to true (default).

    Inputs:
        1. root_path  - full path of a directory
        2. file_types - tuple containing strings of file extensions
            ([file_types=('.jpg', '.png')])
        3. do_sort    - boolean indicating if image path list should be sorted
            ([do_sort=True])

    Output:
        1. list containing full path of all files in the directory and its
            subdirectories

    Example call:
        ROOT_PATH = path/to/some/folder/dude
        all_images = get_image_paths(ROOT_PATH, file_types=('.png'))
    """
    # If the root path points to file, simply return it as a list with 1 element
    if os.path.isfile(root_path):
        return [root_path]
    
    # Gets all images in this root directory
    all_images = []
    for root, _junk, files in os.walk(root_path):
        # Processes files in the root directory - can be transformed into a list
        # comprehension but code might lose clarity
        for file in files:
            if file.lower().endswith(file_types):
                exact_path = os.path.join(root, file)
                all_images.append(exact_path)
    
    # Sort the image path list
    if do_sort:
        all_images.sort()

    return all_images

# ------------------------------------------------------------------------------

def create_dir(dir_path):
  """
  Creates a directory at the specified directory path 'dir_path' IF it does not
  exist. Returns a status of 0 is the directory was successfully created and 
  returns a status of 1 if a directory already exists.

  Inputs:
    1. dir_path - directory path of new directory.
    
  Outputs:
    1. status - 0 to indicate success (directory creation) or 1 to indicate
       failure (directory already exists)
    
  Signature:
    status = create_dir(dir_path)
  """
  # Create directory
  try:
    os.makedirs(dir_path)
    status = 0
  except FileExistsError:
    # Directory already exists
    status = 1

  return status

# ------------------------------------------------------------------------------

def saved_verifier_exists(verifier_name, save_dir=SVD_VRF_DIR):
    """
    Checks if a saved verifier exists with name 'verifier_name'. Does that by
    comparing the 'verifier_name' against the name of all files in 'save_dir'.
    
    Inputs:
        1. verifier_name - string with the name of the verifier.
        2. save_dir - string with the full path of the save directory
            ([save_dir=svd_vrf_dir]).
        
    Output:
        1. Boolean value indicating if saved verifier exists or not.
    
    Signature:
        verifier_exists = saved_verifier_exists(verifier_name,
                                                save_dir=svd_vrf_dir)
    """
    # Save directory provided is not a directory
    if not os.path.isdir(save_dir):
        return False
    
    # Otherwise, check if verifier name is in the names of files in the
    # 'save_dir' directory
    is_in = False
    for file_name in os.listdir(save_dir):
        is_in = is_in or verifier_name in file_name
    
    return is_in

# ------------------------------------------------------------------------------

def save_face_verifiers(verifiers, save_dir=SVD_VRF_DIR, show_prog_bar=True,
                        overwrite=False):
    """
    Saves all face verifier models specified in 'verifiers' using the 'shelve'
    (persistent storage) module. The models are saved in the 'save_dir'
    directory and each model is saved individually. Models that fail to save (or
    produces an error) are skipped. For each model that already exists, this
    function does not overwrite it unless the 'overwrite' flag is set to True.
    
    Inputs:
        1. verifiers - dictionary containing the build face verifier models.
        2. show_prog_bar - boolean that toggles the progress bar on or off.
        3. overwrite - boolean that indicates if the function should overwrite
                        any saved models.
                        
    Outputs:
        1. returns a status flag of True if any model fails to save (otherwise
            returns False)
    
    Signature:
        status = save_face_verifiers(verifiers, show_prog_bar=True,
                    overwrite=False)
    """
    # Checks if the save directory provided is a directory
    if not os.path.isdir(save_dir):
        return True
    
    # Creates the progress bar
    n_verifiers    = len(verifiers)
    disable_option = not show_prog_bar
    pbar           = tqdm(range(0, n_verifiers), desc='Saving verifiers',
                            disable = disable_option)
    
    # Loops through each verifier
    no_errors_flag = False # False means no errors
    for index, verifier_items in zip(pbar, verifiers.items()):
        # Gets the name of the verifier and the verifier object
        name     = verifier_items[0]
        verifier = verifier_items[1]
        
        if not saved_verifier_exists(name, save_dir=save_dir) or overwrite:
            try:
                # Creates the file's full path
                file_fp  = os.path.join(save_dir, name)
        
                # Opens a persistent dictionary, saves the model
                # as a dictionary then closes it
                with shelve.open(file_fp) as d:
                    d[name]  = verifier
            
            except Exception as excpt:
                no_errors_flag = True
            
    return no_errors_flag

# ------------------------------------------------------------------------------

def load_face_verifier(verifier_names, save_dir=SVD_VRF_DIR,
                        show_prog_bar=True, verbose=False):
    """
    Loads all face verifier models specified in 'verifier_names'. Alternatively,
    'all' can be passed as a 'verifier_name' to load all saved models. The
    models are loaded from the 'save_dir' directory.
      
    This function loads each model individually and skips any model that 
    fails to load (or produces an error).
    
    Inputs:
        1. verifier_name - string with the name of the face verifiers.
        2. save_dir      - string with the full path of the save directory
            ([save_dir='']).
        3. show_prog_bar - boolean that toggles the progress bar on or off.
                        
    Outputs:
        1. returns a status flag of True if any model fails to save (otherwise
            returns False)
            
    Signature:
        models = load_face_verifier(verifier_names, save_dir='',
                    show_prog_bar=True)
    """
    # Checks if the save directory provided is a directory
    if not os.path.isdir(save_dir):
        raise OSError(f'Save directory does not exist ({save_dir})!')
    
    # Ensures that the verifier_names is a list (even a single name is provided)
    if not isinstance(verifier_names, list):
        verifier_names = [verifier_names]
        
    # If 'all' was provided, use all model names
    if verifier_names[0].lower() == 'all':
        verifier_names = ['VGG-Face', 'OpenFace', 'Facenet', 'Facenet512',
                          'DeepFace', 'DeepID', 'ArcFace', 'Emotion', 'Age',
                          'Gender', 'Race']
    
    # Creates the progress bar
    n_verifiers    = len(verifier_names)
    disable_option = not show_prog_bar
    pbar           = tqdm(range(0, n_verifiers), desc='Loading verifiers',
                            disable = disable_option)
    
    models = {}
    for index, verifier_name in zip(pbar, verifier_names):       
        # Checks if the face verifier model exists
        if saved_verifier_exists(verifier_name, save_dir=save_dir):
            # Loads the model
            file_fp = os.path.join(save_dir, verifier_name)
            with shelve.open(file_fp) as model:
                if verbose:
                    print(f'[load_face_verifier] Loading model',
                          f'{verifier_name}: ', end='')
                try:
                    models[verifier_name] = model[verifier_name]
                    if verbose:
                        print('success!')
                except Exception as excpt:
                    if verbose:
                        print(f'failed! Reason: {excpt}\n',
                               '[load_face_verifier] Attempting to build',
                               ' & save model from scratch: ', sep='', end='')
                    try:
                        cur_model = build_verifier(verifier_name)
                        models[verifier_name] = cur_model
                        save_face_verifiers(cur_model, save_dir=save_dir,
                                            show_prog_bar=False, overwrite=True)
                        if verbose:
                            print('success!')
                    except Exception as excpt:
                        if verbose:
                            print(f'failed! Reason: {excpt}')
                                
        else:
            # Otherwise, tries to build model from scratch & save it
            if verbose:
                print(f'[load_face_verifier] Model {verifier_name} ',
                      'does not exist.', '\nAttempting to build & save model ',
                      'from scratch: ', sep='', end='')
            try:
                cur_model = build_verifier(verifier_name)
                models[verifier_name] = cur_model
                save_face_verifiers(cur_model, save_dir=save_dir,
                                    show_prog_bar=False, overwrite=False)
                if verbose:
                    print('success!')
            except Exception as excpt:
                if verbose:
                    print(f'failed! Reason: {excpt}')
            
    return models

# ------------------------------------------------------------------------------
        
def load_representation_db(file_path, verbose=False):
    """
    Loads a database (at 'file_path') containing representations of face images.
    The database is assumed to be a pickled Python object. The database is
    expected to be a list of Representation object (see help(Representation)
    for more information). If verbose is set to True, the loading processing is
    printed, with any errors being reported to the user.

    Inputs:
        1. file_path - string with file's full path
        2. verbose - flag indicating if the function should print information
            about the loading process and errors ([verbose=False])

    Output:
        1. database object (list of Representation objects)
    
    Signature:
        db = load_representation_db(file_path, verbose=False)
    """
    # Prints message
    if verbose:
        print('Opening database: ', end='')

    # Checks if path provided points to a valid database
    if os.path.isfile(file_path):
        # Try to open pickled database (list of objects)
        try:
            db = pickle.load(open(file_path, 'wb'))
            if verbose:
                print('success!')
        
        except (OSError, IOError) as e:
            if verbose:
                print(f'failed! Reason: {e}')
            db = []

    # If path does not point to a file, open an 'empty' database
    else:
        print(f'failed! Reason: database does not exist.')
        db = []

    return db
                
# ------------------------------------------------------------------------------

def load_representation_db(file_path, verbose=False):
    """
    Loads a database (at 'file_path') containing representations of face images.
    The database is assumed to be a pickled Python object. The database is
    expected to be a list of Representation object (see help(Representation)
    for more information). If verbose is set to True, the loading processing is
    printed, with any errors being reported to the user.

    Inputs:
        1. file_path - string with file's full path
        2. verbose - flag indicating if the function should print information
            about the loading process and errors ([verbose=False])

    Output:
        1. database object (list of Representation objects)
    
    Signature:
        db = load_representation_db(file_path, verbose=False)
    """
    # Prints message
    if verbose:
        print('Opening database: ', end='')

    # Checks if path provided points to a valid database
    if os.path.isfile(file_path):
        # Try to open pickled database (list of objects)
        try:
            db = pickle.load(open(file_path, 'wb'))
            if verbose:
                print('success!')
        
        except (OSError, IOError) as e:
            if verbose:
                print(f'failed! Reason: {e}')
            db = []

    # If path does not point to a file, open an 'empty' database
    else:
        print(f'failed! Reason: database does not exist.')
        db = []

    return db

# ------------------------------------------------------------------------------

def detect_faces(img_path, detector_backend = 'opencv', align = True,
                 return_type = 'both', face_detector = None):
    """
    Detects faces in an image (and optionally aligns them).
    
    Inputs:
        1. img_path - image path, base64 image or numpy array image
        2. detector_backend - string corresponding to detector ([opencv],
            ssd, dlib, mtcnn, retinaface, mediapipe).
        3. align - flag indicating if face should be aligned ([align=True]).
        4. return_type - string indicating if faces, regions or both should
            be returned ('faces', 'regions', ['both']).
            
    Outputs:
        If return_type='regions':
            Dictionary containing list of face detections. The face detections
            (or regions of interest - rois) are lists with the format
            [top-left x, top-left y, width, height]. The dictionary key is
            'regions'.
            
        If return_type='faces':
            Dictionary containing list of detected faces. Each detection is an
            image with 'target_size' size (the number of color channels is
            unchanged). The dictionary key is 'faces'.
            
        If return_type='both':
            Dictionary containing both of the above outputs: face detections and
            detected faces. The dictionary keys are 'faces' and 'regions'. 
    
    Signature:
        output = detect_faces(img_path, detector_backend = 'opencv',
                              align = True, return_type = 'both')
    """
    # Raises an error if return type is not 'faces', 'regions' or 'both'.
    # Otherwise, initializes lists.
    if return_type == 'faces' or return_type == 'regions' or \
        return_type == 'both':
        faces = []
        rois  = []
    else:
        raise ValueError("Return type should be 'faces', 'regions' or 'both'.")
    
    # Loads image. Image might be path, base64 or numpy array. Convert it to numpy
    # whatever it is.
    img = functions.load_image(img_path)

    # The detector is stored in a global variable in FaceDetector object.
    # This call should be completed very fast because it will return found in
    # memory and it will not build face detector model in each call (consider for
    # loops)
    if face_detector == None:
        face_detector = FaceDetector.build_model(detector_backend)
    detections = FaceDetector.detect_faces(face_detector, detector_backend,
                                            img, align)

    # Prints a warning and returns an empty dictionary and error if no faces were
    # found, otherwise processes faces & regions
    if len(detections) == 0:
        print('Face could not be detected or the image contains no faces.')

    else:
        # Loops through each face & region pair
        for face, roi in detections:

            # Only process images (faces) if the return type is 'faces' or 'both'
            if return_type == 'faces' or return_type == 'both':
                # Appends processed face
                faces.append(face)
    
            # Only process regions (rois) if the return type is 'regions' or 'both'
            if return_type == 'regions' or return_type == 'both':
                rois.append(roi)

  # ------------------------
  
    if return_type == 'faces':
        return {'faces':faces}
    elif return_type == 'regions':
        return {'regions':rois}
    else:
        assert return_type == 'both', "Return type should be 'both' here."
        return {'faces':faces, 'regions':rois}

# ------------------------------------------------------------------------------

def process_face(img_path, target_size=(224, 224), normalization='base',
                    grayscale=False):
    """
    Applies some processing to an image of a face:
        1. Loads the image from an image path, base64 encoding or numpy array
        2. If the 'grayscale' flag is True, converts the image to grayscale
        3. Resizes the image based on the smallest factor (target_size /
            dimension) and zero pads the resized image to match the target size.
        4. If for some reason the image is still not the target size, resizes
            the modified image once again.
        5. Normalizes the image based on the normalization option:
            a. 'base': do nothing
            b. 'raw': restore input in scale of [0, 255]
            c. 'Facenet': 'raw' then subtract image mean and divide by image
                    standard deviation
            d. 'Facenet2018': 'raw' then divide by 127.5 and subtract 1
            e. 'VGGFace': 'raw' then mean subtraction based on VGGFace1 training
                    data
            f. 'VGGFace2': 'raw' then mean subtraction based on VGGFace2
                    training data
            g. 'ArcFace': based on a reference study, 'raw', then pixels are
                    normalized by subtracting 127.5 then dividing by 128.

    Inputs:
        1. img_path - image path, base64 image or numpy array
        2. target_size - tuple containing desired X and Y dimensions
            ([target_size=(224, 224)])
        3. normalization - defines a type of normalization
            ([normalization='base'])
        4. grayscale - flag indicating if image should be converted to grayscale
            ([grayscale=False])

    Output:
        1. processed image as a numpy array

    Signature:
        face_image = process_face(img_path, target_size=(224, 224),
                                  normalization='base', grayscale=False)
    """
    # Loads the face image. Image might be path, base64 or numpy array. Convert
    # it to numpy whatever it is.
    face = functions.load_image(img_path)
    
    # Ensures that both dimensions are >0, otherwise raises error
    if face.shape[0] == 0 or face.shape[1] == 0:
        raise ValueError(f'Detected face shape is {face.shape}.')

    # Converts to grayscale if face is 
    if grayscale:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                
    # Resizes face
    if face.shape[0] > 0 and face.shape[1] > 0:
        factor_0 = target_size[0] / face.shape[0]
        factor_1 = target_size[1] / face.shape[1]
        factor   = min(factor_0, factor_1)

    dsize  = (int(face.shape[1] * factor), int(face.shape[0] * factor))
    face   = cv2.resize(face, dsize)

    # Then pad the other side to the target size by adding black pixels
    diff_0 = target_size[0] - face.shape[0]
    diff_1 = target_size[1] - face.shape[1]
                
    if not grayscale:
        # Put the base image in the middle of the padded image
        face = np.pad(face, ((diff_0 // 2, diff_0 - diff_0 // 2),
                             (diff_1 // 2, diff_1 - diff_1 // 2), (0, 0)),
                             'constant')
    else:
        face = np.pad(face, ((diff_0 // 2, diff_0 - diff_0 // 2),
                             (diff_1 // 2, diff_1 - diff_1 // 2)),
                             'constant')

    # Double check if target image is not still the same size with target.
    if face.shape[0:2] != target_size:
        face = cv2.resize(face, target_size)

    # Normalizing the image pixels
    if normalization == 'base':
        face  = image.img_to_array(face) #what this line doing? must?
        face  = np.expand_dims(face, axis = 0)
        face /= 255 # normalize input in [0, 1]
    else:
        face = functions.normalize_input(face, normalization=normalization)
    
    return face

# ------------------------------------------------------------------------------

def calc_cosine_similarity(A, B):
    """
    Calculates the cosine similarity metric between matrices A and B. If A is a
    vector, it is converted into a matrix so that the cosine metric can be
    calculated normally.

    Inputs:
        1. A - N x M matrix with N embeddings with M elements
        2. A - I x M matrix with I embeddings with M elements

    Outputs:
        1. matrix of cosine similarity metric between A and B

    Signature:
        csm = calc_cosine_similarity(A, B)
    """
    if A.ndim == 1:
        A = A[np.newaxis, :]

    num = np.dot(A, B.T)
    p1  = np.sqrt(np.sum(A**2, axis=1))[:, np.newaxis]
    p2  = np.sqrt(np.sum(B**2, axis=1))[np.newaxis, :]
    return 1 - (num / (p1 * p2))

# ------------------------------------------------------------------------------

def calc_euclidean_similarity(A, B, l2_normalize=False):
    """
    Calculates the Euclidean similarity metric between matrices A and B. If A is
    a vector, it is converted into a matrix by repeating (and stacking) it
    horizontally until it has the correct dimensions. If 'l2_normalize' is set
    to True, then the function applies L2 normalization to the inputs before
    calculating the Euclidean similarity (distance).

    Inputs:
        1. A - N x M matrix with N embeddings with M elements
        2. A - I x M matrix with I embeddings with M elements
        3. l2_normalize - boolean to indicate if the inputs should be L2
            normalized before calculating the similarity ([l2_normalize=True])

    Outputs:
        1. matrix of Euclidean similarity metric between A and B

    Signature:
        edm = calc_euclidean_similarity(A, B, l2_normalize=False)
    """
    # Applies l2 normalization to the inputs if necessary
    if l2_normalize:
        if A.ndim == 1:
            A = A / np.sqrt(np.sum(np.multiply(A, A)))
        else:
            A = np.transpose(A.T / np.linalg.norm(A, axis=1))

        if B.ndim == 1:
            B = B / np.sqrt(np.sum(np.multiply(B, B)))
        else:
            B = np.transpose(B.T / np.linalg.norm(B, axis=1))

    # 'Repeats vertically' vector A until it is a matrix with appropriate
    # dimensions 
    if A.ndim == 1:
        A = np.tile(A, (B.shape[0], 1))

    # Calcultes and returns the Euclidean distance
    return np.sqrt(np.sum((A - B) * (A - B), axis=1))

# ------------------------------------------------------------------------------

def calc_similarity(tgt_embd, embds, metric='cosine', model_name='VGG-Face',
                    threshold=-1):
    """
    Calculates the similarity (distance) between both embeddings ('tgt_embd'
    and 'embds') using the 'metric' distance metric. If the 'threshold' < 0 then
    it is automatically determined based on the 'model_name' provided. If a 
    custom threshold is specified, then the 'model_name' input is unused.

    Note that 'embds' can be a N x M matrix (N embeddings each with M elements)
    and 'tgt_embd' can only be a 1 x M embedding.

    Inputs:
        1. tgt_embd - 1-D numpy array containing the embedding.
        2. embds - 1-D or 2-D numpy array containing the embedding(s).
        3. metric - string specifying the distance metric to be used
            (['cosine'], 'euclidean', 'l2_euclidean').
        4. model_name - string specifying the model name (['VGG-Face']).
        5. threshold - if a negative float is provided, then the threshold is
            calculated automatically based on the model name provided.
            Otherwise, the threshold provided is used ([threshold=-1]).

    Output:
        1. dictionary containing the indexes of matches (key: idxs), the
            threshold value used (key: threshold) and the distances (using the
            specified metric) of the matches (key: distances). Note that if no
            match is found, then the 'indexes' will have a length of zero (i.e.
            will be empty).
    
    Signature:
        similarity_obj = calc_similarity(tgt_embd, embds, metric='cosine',
                                         model_name='VGG-Face', threshold=-1)
    """
    # Calculates the distance based on the metric provided, otherwise raises a
    # value error
    if metric == 'cosine':
        distances = calc_cosine_similarity(tgt_embd, embds)
    elif metric == 'euclidean':
        distances = calc_euclidean_similarity(tgt_embd, embds)
    elif metric == 'euclidean_l2':
        distances = calc_euclidean_similarity(tgt_embd, embds,
                                              l2_normalize=True)
    else:
        raise ValueError(f'Invalid metric passed: {metric}')

    # If threshold is negative, determine threshold automatically based on the
    # model name and distance metric
    if threshold < 0:
        threshold = dst.findThreshold(model_name, metric)
    
    # Processes distances
    distances = distances.flatten() # flattens into 1-D array

    # Makes a decision
    decision  = (distances <= threshold)
    idxs      = np.where(decision)[0]

    # Determines the corresponding distances of each match (decision==True)
    dists = []
    j     = 0
    for i in range(len(distances)):
        if i in idxs:
            dists.append(distances[i])
            j += 1
        
        if j == len(idxs):
            break

    # Sort the indexes by smallest distance
    srt_dists = []
    srt_idxs  = []
    for dist, idx in sorted(zip(dists, idxs)):
        srt_idxs.append(idx)
        srt_dists.append(dist)

    return {'idxs': np.array(srt_idxs),
            'threshold': threshold,
            'distances': np.array(srt_dists)}

# ------------------------------------------------------------------------------

def calc_embedding(img_path, verifier_models, detector_name='opencv',
                    align=True, verifier_names='VGG-Face',
                    normalization='base'):
    """
    TODO: ADD DESCRIPTION
    """
    # Converts verifier names into a list if it is a single entry
    if not isinstance(verifier_names, list):
        verifier_names = [verifier_names]

    # Tries to detect faces & align:
    try:
        output = detect_faces(img_path, detector_backend=detector_name,
                                align=align, return_type='both',
                                face_detector=None)
    except:
        print('[calc_embedding] Error: face detection failed!')
        return ([], {})

    # TODO: MAKE THE FUNCTION ACCEPT MULTIPLE FACES IN ONE IMAGE
    # Since we assume there is only 1 face (and region):
    face   = output['faces'][0]
    region = output['regions'][0]

    # For each verifier model provided
    embeddings={}
    for verifier_name in verifier_names:
        # Gets the current verifier model
        model = verifier_models[verifier_name]

        # Determine target size
        input_x, input_y = functions.find_input_shape(model)

        # Process face
        processed_face = process_face(face, target_size=(input_x, input_y),
                                normalization=normalization, grayscale=False)

        # Calculate embeddings
        embeddings[verifier_name] = model.predict(processed_face)[0]

    return (region, embeddings)

# ------------------------------------------------------------------------------

def create_new_representation(img_path, region, embeddings, tag='', uid='',
                                ignore_taglist=['--', '---']):

    # If Unique IDentifier (UID) is not provided, generate one
    if len(uid) == 0 or uid == '':
        uid = uuid4()

    # If tag is in the ignore taglist, then it is considered as a "ignore" tag
    if tag in ignore_taglist:
        tag = ''

    # Returns the new representation
    return Representation(uid, image_name=img_path.split('/')[-1], 
                          image_fp=img_path, name_tag=tag, region=region,
                          embeddings=embeddings)

# ------------------------------------------------------------------------------

def get_embeddings_as_array(db, verifier_name):
    """
    Gets all of the embeddings for a given 'verifier_name' from the database
    'db' and returns it as a N x M numpy array where N is the number of
    embeddings and M is the number of elements of each embeddings.

    Inputs:
        1. db - database object (list of Representation objects)
        2. verifier_name - name of verifier

    Output:
        1. N x M numpy array where each row corresponds to a face image's
            embedding

    Signature:
        embeddings = get_embeddings_as_array(db, verifier_name)
    """
    embeddings = [] # initializes empty list
    for rep in db:
        embeddings.append(rep.embeddings[verifier_name])

    return np.array(embeddings)

# ------------------------------------------------------------------------------

def person_name_lookup(img_paths):
    """
    
    """
    name_list = []
    for pth in img_paths:
        cur_num = int(pth.split('/')[-1][3:5])

        if cur_num   in [ 1,  2,  4,  5,  6,  7, 10, 11]:   # Angelina Jolie
            name_list.append('Angelina Jolie')
        
        elif cur_num in [ 3, 12, 53, 54, 55, 56]:           # Jennifer Aniston
            name_list.append('Jennifer Aniston')

        elif cur_num in [ 8,  9, 47, 48, 49, 50, 51]:       # Scarlett Johansson
            name_list.append('Scarlett Johansson')

        elif cur_num in [13, 14, 15, 57, 58]:               # Mark Zuckerberg
            name_list.append('Mark Zuckerberg')

        elif cur_num in [16, 17, 59, 61, 62]:               # Jack Dorsey
            name_list.append('Jack Dorsey')

        elif cur_num in [18, 19, 67]:                       # Elon Musk
            name_list.append('Elon Musk')

        elif cur_num in [20, 21]:                           # Jeff Bezos
            name_list.append('Jeff Bezos')

        elif cur_num in [22, 23]:                           # Marissa Meyer
            name_list.append('Marissa Meyer')

        elif cur_num in [24, 25]:                           # Sundar Pichai
            name_list.append('Sundar Pichai')
        
        elif cur_num in [26, 27, 28, 42, 43, 44, 45, 46]:   # Katy Perry
            name_list.append('Katy Perry')

        elif cur_num in [29, 30, 31, 32, 33]:               # Matt Damon
            name_list.append('Matt Damon')

        elif cur_num in [34, 35, 36, 37]:                   # Leonardo DiCaprio
            name_list.append('Leonardo DiCaprio')

        elif cur_num in [38, 39, 40, 41]:                   # George Clooney
            name_list.append('George Clooney')

        else:
            name_list.append('')                            # missing name

    return name_list

# ------------------------------------------------------------------------------

def create_reps_from_dir(img_dir, verifier_models, detector_name='opencv',
                    align=True, verifier_names='VGG-Face', show_prog_bar=True,
                    normalization='base', tags=[], uids=[], skip_get_imgs=False,
                    verbose=False):
    """
    Creates a representations from images in a directory 'img_dir'. The
    representations are returned in a list, and the list of representations. If
    tags and/or unique identifiers (uids) are provided, make sure that they 
    correspond to the sorted (ascending) image names contained in 'img_dir'.

    Inputs:
        1. img_dir - string with the full path to the directory containing the
            images.

        2. verifier_models - dictionary containing model name (key) and model
            object (value).

        3. detector_name - string with the face detector name ([opencv], ssd,
            dlib, mtcnn, retinaface).
        
        4. align - boolean flag to indicate if face images should be aligned.
            Improves face recognition performance at the cost of some speed
            ([True], False).

        5. verifier_names - string with the face verifier name ([VGG-Face],
            OpenFace, Facenet, Facenet512, DeepFace, DeepID, ArcFace).

        6. show_prog_bar - boolean that toggle the progress bar on or off
            ([True], False).

        7. normalization - normalizes the face image and may increase face
            recognition performance depending on the normalization type and the
            face verifier model ([base], raw, Facenet, Facenet2018, VGGFace,
            VGGFace2, ArcFace).

        8. tags - list of strings where each string corresponds to a tag for the
            i-th image, i.e. tags[0] is the tag for the first image in the
            sorted list of image name obtain from 'img_dir' directory. If an
            empty list is provided, this is skipped during the representation
            creation process ([tags=[]]).

        9. uids - list of strings where each string corresponds to a unique
            identifier (UID) for the i-th image, i.e. uids[0] is the UID for the
            first image in the sorted list of image name obtain from 'img_dir'
            directory. If an empty list is provided, a UID is created for each
            image during the representation creation process ([uids=[]]).

        10. skip_get_imgs - boolean to skip getting the image paths from the
            directory. This should be set to True if a list of image paths is
            provided as 'img_dir' as opposed to a directory
            ([skip_get_imgs=False]).
            
        11. verbose - boolean to toggle function warnings and other messages
            ([True], False).

        Note: the 'tags' and 'uids' lists (inputs 8 and 9) must have the same
        number of elements (length) and must match the number of images in
        'img_dir'. If not, these inputs will be treated as empty lists (i.e.
        ignored).

    Outputs:
        1. list of Representation objects. For more information about the
            Representation class attributes and methods, use
            help(Representation)

    Signature:
        rep_db = create_reps_from_dir(img_dir, verifier_models, 
                        detector_name='opencv', align=True,
                        verifier_names='VGG-Face', show_prog_bar=True,
                        normalization='base', tags=[], uids=[], verbose=False)
    """
    # Initializes skip flags and database (list of Representation objects)
    skip_tag = False
    skip_uid = False
    rep_db   = []
    
    # Assuming img_dir is a directory containing images
    if not skip_get_imgs:
        img_paths = get_image_paths(img_dir)
        img_paths.sort()
    else:
        img_paths = img_dir

    # No images found, return empty database
    if len(img_paths) == 0:
        return []

    # If tags list does not have the same number of elements as the images (i.e.
    # 1 tag per image), ignore it
    if len(tags) != len(img_paths):
        if verbose:
            print('[create_reps_from_dir] Number of tags and image paths',
                  'must match. Ignoring tags list.')
        skip_tag = True

    # If uids list does not have the same number of elements as the images (i.e.
    # 1 UID per image), ignore it
    if len(uids) != len(img_paths):
        if verbose:
            print('[create_reps_from_dir] Number of UIDs and image paths',
                  'must match. Ignoring uids list.')
        skip_uid = True

    # Creates the progress bar
    n_imgs  = len(img_paths)
    disable = not show_prog_bar
    pbar    = tqdm(range(0, n_imgs), desc='Creating representations',
                    disable=disable)

    # Loops through each image in the 'img_dir' directory
    for pb_idx, i, img_path in zip(pbar, range(0, n_imgs), img_paths):
        # Calculate the face image embedding
        region, embeddings = calc_embedding(img_path, verifier_models,
                                            align=align,
                                            detector_name=detector_name, 
                                            verifier_names=verifier_names,
                                            normalization=normalization)

        # Determines if tag was provided and should be used when creating this
        # representation
        if skip_tag:
            tag = ''
        else:
            tag = tags[i]

        # Determines if UID was provided and should be used when creating this
        # representation
        if skip_uid:
            uid = ''
        else:
            uid = uids[i]

        # Create a new representation and adds it to the database
        rep_db.append(create_new_representation(img_path, region, embeddings,
                                                tag=tag, uid=uid))

    # Return representation database
    return rep_db

# ------------------------------------------------------------------------------

def find_image_in_db(img_path, db, shortcut=None):
    """
    Finds the corresponding Representation in the database 'db'. Uses the
    image's name obtained from its full path ('img_path') to match the
    Representation to the image. An optional dictionary ('shortcut') can be
    provided where each letter corresponds to an index to speed up the search.
    Consider a sorted database where the first image with a name starting with
    a 'g' is at position 156. Then a shortcut dictionary with the key/value pair
    'g':155 will ensure this function starts at the position 156 and wont waste
    time previous database entries.

    Inputs:
        1. img_path - image path, base64 image or numpy array
        2. db - database object (list of Representation objects)
        3. shortcut - optional dictionary with letter/index key/value pairs
            ([shortcut=None])

    Output:
        1. tuple containing the matching Representation object and the index
            corresponding to that object in the database (i.e. output
            representation == database[output index]). If no match is found,
            both representation and index are returned as empty lists. If
            multiple entries are found, multiple Representations and indexs are
            returned as lists.
    """
    rep_objs = [] # empty representation object
    rep_idxs = [] # no matching index

    # Database is empty
    if len(db) == 0:
        pass # do nothing

    # 'shortcut' dictionary is provided
    elif not shortcut == None:
        # Get image name
        img_name = img_path.split('/')[-1].lower()
        idx      = shortcut[img_name[0]]

        # 
        for i, rep in enumerate(db[idx::]):
            if img_name.lower() == rep.image_name:
                rep_objs.append(rep)
                rep_idxs.append(i + idx)

    # 'shortcut' dictionary is not provided
    else:
        # Get image name
        img_name = img_path.split('/')[-1].lower()

        # 
        for i, rep in enumerate(db):
            if img_name.lower() == rep.image_name:
                rep_objs.append(rep)
                rep_idxs.append(i)

    return (rep_objs, rep_idxs)

# ------------------------------------------------------------------------------

def create_faiss_index(embeddings, metric='cosine'):
    """
    Create faiss indexes from 'embeddings'. Embeddings are expected to be a
    N x M numpy array with N embeddings each with M elements.

    Inputs:
        1. embeddings - N x M numpy array
        2. metric - string specifying the distance/similarity metric ([cosine],
            euclidean, euclidean_l2)

    Output:
        1. index object with embeddings added

    Signature:
        index = create_faiss_index(embeddings, metric='cosine')
    """
    # Create the appropriate index based on the distance metric provided. If the
    # distance metric provided is not valid (not cosine, euclidean or
    # euclidean_l2), an error is raised.
    if metric == 'cosine':
        index = faiss.IndexFlatIP(embeddings.shape[1])
    elif metric in ['euclidean', 'euclidean_l2']:
        index = faiss.IndexFlatL2(embeddings.shape[1])
    else:
        raise ValueError(f'{metric} is an invalid metric!',
                          'Expected cosine, euclidean or euclidean_l2.',
                          sep='\n')
    
    # Applies L2 normalization in the cases of cosine or euclidean_l2 metrics
    if metric in ['cosine', 'euclidean_l2']:
        embeddings = dst.l2_normalize(embeddings)

    # Adds the embeddings to the index object
    index.add(embeddings)

    return index

# ------------------------------------------------------------------------------

def verify_with_faiss(target_embd, index, db, top_x=3, model_name='VGG-Face',
                      metric='cosine', threshold=-1):
    """
    Uses the Faiss library to speed up the verification process. The 'index'
    object is a faiss-specific object with the embeddings of the whole database
    'db'. Using faiss fast search, the 'top_x' nearest neighbour embeddings are
    obtained and are used to create a subset of the original database.

    The similarity (and matches) are then calculated using the subset database,
    which is only 'top_x' large. Note that the speed gains are more noticeable
    for larger databases and small values of 'top_x'. Also note that if there
    are more than 'top_x' matches, then only 'top_x' will be returned.

    Finally, if top_x = 1 then this function returns the nearest neighbour match
    and the 'model_name' and 'threshold' parameters have no effect. In this
    case, the threshold value in the output dictionary will be equal to -1 to
    indicate it has not been used.

    Inputs:
        1. target_embd - 1-D numpy array containing the embedding.
        2. index - faiss-specific index object. Use 'load_faiss_indexes'
                function to create it.
        3. db - database (list of Representation objects)
        4. top_x - natural number indicating the number of nearest neighbours to
                filter the database ([top_x=3])
        5. model_name - string specifying the model name (['VGG-Face']).
        6. metric - string specifying the distance metric to be used
                (['cosine'], 'euclidean', 'l2_euclidean').
        7. threshold - if a negative float is provided, then the threshold is
                calculated automatically based on the model name provided.
                Otherwise, the threshold provided is used ([threshold=-1]).
    
    Output:
        1. dictionary containing the indexes of matches (key: idxs), the
            threshold value used (key: threshold) and the distances calculated
            using the specified metric (key: distances). Note that if no match
            is found, then the 'indexes' will have a length of zero (i.e. will
            be empty). The indexes and distances are sorted so that the first
            index has the smallest distance, the second index has the second
            smallest distance, and so on.

    Signature:
        similarity_obj = verify_with_faiss(target_embd, index, db, top_x=3,
                                    model_name='VGG-Face', metric='cosine',
                                    threshold=-1)
    """
    # Obtains the 'top_x' nearest neighbours
    distances, neighbors = index.search(target_embd, top_x)

    # If top_x is 1 then simply return the closest neighbour
    if top_x == 1:
        similarity_obj['idxs']      = np.array(neighbors[0], dtype=int)
        similarity_obj['threshold'] = -1
        similarity_obj['distances'] = distances
        return similarity_obj

    # Obtains the embeddings for the 'filtered' (top_x) representations
    idxs          = []
    filtered_reps = []
    for i in np.array(neighbors[0], dtype=int):
        idxs.append(i)
        filtered_reps.append(db[i])

    embds = get_embeddings_as_array(filtered_reps, model_name)

    # Calculates the matches to the target embedding (from the top_x filtered
    # set of representations)
    similarity_obj = calc_similarity(target_embd, embds, metric=metric,
                                     model_name=model_name, threshold=threshold)

    # Note that the indexes returned in the similarity object refer to the
    # filtered representations. So the correct indexes (from the original
    # database) are actually the idxs[similarity_obj['idxs']]
    similarity_obj['idxs'] = [idxs[i] for i in similarity_obj['idxs']]

    return similarity_obj

# ------------------------------------------------------------------------------


In [4]:
# All detectors and verifiers
DETECTOR_NAMES = ('opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface')
VERIFIER_NAMES = ('VGG-Face', 'OpenFace', 'Facenet', 'Facenet512', 'DeepFace',
                  'DeepID', 'Dlib', 'ArcFace', 'Emotion', 'Age', 'Gender',
                  'Race')

# Directories & paths initialization
print('  -> Directory creation:')
for directory in directory_list:
    print(f'Creating {directory} directory: ', end='')
    
    if create_dir(directory):
        print('directory exists. Continuing...')
    else:
        print('success.')
print('')

  -> Directory creation:
Creating /home/rpessoa/projects/loki/api directory: directory exists. Continuing...
Creating /home/rpessoa/projects/loki/api/data directory: directory exists. Continuing...
Creating /home/rpessoa/projects/loki/api/data/raw directory: directory exists. Continuing...
Creating /home/rpessoa/projects/loki/api/data/targets directory: directory exists. Continuing...
Creating /home/rpessoa/projects/loki/api/data/database directory: directory exists. Continuing...
Creating /home/rpessoa/projects/loki/api/saved_models directory: directory exists. Continuing...
Creating /home/rpessoa/projects/loki/api/saved_models/verifiers directory: directory exists. Continuing...



### Demo

In [221]:
chosen_model   = 'ArcFace'
chosen_backend = 'retinaface'
align          = True
normalization  = 'base'
metric         = 'cosine'
use_faiss      = True
top_x          = 10
threshold      = -1 # automatically get threshold based on chosen model & metric

# Paths
dataset_path      = ''
stored_imgs_path  = '/' + os.path.join(*(API_DIR.split('/')[:-1]), 'dataset')
db_path           = '' # no database exists
svd_verifier_path = SVD_VRF_DIR

# Splits
db_split  = [ 5,  6,  8, 11, 12, 14, 17, 19, 20, 23, 24, 26, 27, 28, 29, 30,\
             32, 35, 36, 38, 41, 51, 52, 55, 60, 61]
tgt_split = [ 1,  2,  3,  4,  7,  9, 10, 13, 15, 16, 18, 21, 22, 25, 31, 33,\
             34, 37, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54,\
             56, 57, 58, 59]


We will start this demo by downloading a small dataset to test the FR system. This dataset is provided in the deepface repository and contains varying numbers of images of 13 different celebraties.

Then, the dataset is divided into a database set and a target set. The database set will be used to create an initial database to run the recognition of the target set images against.

In [ ]:
# Code to download the dataset in case 'dataset_path' does not exist

In [81]:
# Obtain the database and target splits' image paths & tags
all_imgs = get_image_paths(stored_imgs_path)   # gets all image paths
all_tags = person_name_lookup(all_imgs)        # gets the tag for each image

db_imgs  = [all_imgs[i-1] for i in db_split]   # gets all database images
tgt_imgs = [all_imgs[i-1] for i in tgt_split]  # gets all target images

db_tags  = [all_tags[i-1] for i in db_split]   # gets all database tags
#tgt_tags = [all_tags[i-1] for i in tgt_split]  # gets all target tags

Now we will load all the relevant face verifier models. To keep things simple, we will only use one.

In [84]:
# Loading just the ArcFace model as this model will be used throughout this demo
models = load_face_verifier([chosen_model], save_dir=svd_verifier_path,
                            show_prog_bar=False, verbose=True)

[load_face_verifier] Loading model ArcFace: success!

 {'ArcFace': <keras.engine.functional.Functional object at 0x7f06b17e8580>}


Now, we will create the database from the *'db_imgs'* image set.

In [96]:
# Creates the representation database from the image paths
rep_db = create_reps_from_dir(db_imgs, models, detector_name=chosen_backend,
                align=align, verifier_names=chosen_model, show_prog_bar=True,
                normalization=normalization, tags=db_tags, uids=[],
                skip_get_imgs=True, verbose=False)

# 
print('\nCreating faiss indexes for faster search: ', end='')
embds  = get_embeddings_as_array(rep_db, chosen_model)
findex = create_faiss_index(embds, metric=metric)
print('done!\n')

# - Print database summary
for i in range(len(rep_db)):
    print(f'Tag: {rep_db[i].name_tag}'.ljust(25),
          f'Path: {rep_db[i].image_fp}',
          f'UID: {rep_db[i].unique_id}', sep='    |   ')

Creating representations: 100%|██████████| 26/26 [02:25<00:00,  5.61s/it]


 ---------------------------------------------------------------------------------------------------- 
Creating faiss indexes for faster search: done!
 ---------------------------------------------------------------------------------------------------- 

Tag: Angelina Jolie          |   Path: /home/rpessoa/projects/loki/dataset/img05.jpg    |   UID: 059a8fbf-cb26-49be-a29d-8d4c66fbb2f6
Tag: Angelina Jolie          |   Path: /home/rpessoa/projects/loki/dataset/img06.jpg    |   UID: 228db835-934c-4399-a3c5-352a7141fb71
Tag: Scarlett Johansson      |   Path: /home/rpessoa/projects/loki/dataset/img08.jpg    |   UID: 99e30e90-727b-4390-be4e-3b620fcdf67f
Tag: Angelina Jolie          |   Path: /home/rpessoa/projects/loki/dataset/img11.jpg    |   UID: 372c96eb-bae3-4283-b2cc-2b4bdf002436
Tag: Jennifer Aniston        |   Path: /home/rpessoa/projects/loki/dataset/img12.jpg    |   UID: e5f005ca-a6be-4034-9d3b-f86fbf760111
Tag: Mark Zuckerberg         |   Path: /home/rpessoa/projects/loki/dataset

We will now verify each target image against the entire database.

In [222]:
if use_faiss:
    print('Verifying using FAISS:')
else:
    print('Verifying without FAISS:')

t_start = timer()   # initializes timer
tgt_embeddings = [] # initializes target embeddings

for i, pth in zip(range(len(tgt_imgs)), tgt_imgs):
    # Check if a representation exists for the current image in the database
    # NOTE: This is just an 'illustrative' step as all target images are new
    rep, idx = find_image_in_db(pth, rep_db, shortcut=None) # for now, no representation exists

    # Calculate the face image embedding and stores it
    region, embedding = calc_embedding(pth, models, align=align,
                                        detector_name=chosen_backend, 
                                        verifier_names=chosen_model,
                                        normalization=normalization)
    embedding = embedding[chosen_model] # since we are using 1 model, there will
                                        # be a single embedding
    tgt_embeddings.append(embedding)

    # Calculates the similarity between the current embedding and the database
    if use_faiss:
        similarity_obj = verify_with_faiss(embedding[None, :], findex, rep_db,
                                        top_x=top_x, model_name=chosen_model,
                                        metric=metric, threshold=threshold)
    else:
        similarity_obj = calc_similarity(embedding, embds, metric=metric,
                                   model_name=chosen_model, threshold=threshold)

    # print('Current pth: {}'.format(pth),
    #       'sim1: {}'.format(similarity_obj['idxs']),
    #       'sim2: {}'.format(similarity_obj2['idxs']), sep='  |  ')

    # Gets all matches
    matches = [rep_db[i] for i in similarity_obj['idxs']]
    
    # Prints current iteration, image information and verification result
    print('Target {}: {}'.format(i, pth.split('/')[-1]).ljust(22), end='')

    if len(matches) > 0:
        print(' (verified: True  | name: {})  |  (found {} matches)'.format(
                                    matches[0].name_tag, len(matches)))
    else:
        print(' (verified: False | name: ---)  |  (found 0 matches)')


t_end = timer()
print('\n', '-' * 100)
print('Verifying all ({}) images'.format(len(tgt_imgs)),
      'took {:.3f} seconds'.format(t_end - t_start),
      '(approx {:.3f} s/image)'.format((t_end - t_start) / len(tgt_imgs)))

Verifying using FAISS:
Current pth: /home/rpessoa/projects/loki/dataset/img01.jpg  |  sim1: [0, 3, 1]  |  sim2: [23 22  4]
Target 0: img01.jpg    (verified: True  | name: Jennifer Aniston)  |  (found 3 matches)

Current pth: /home/rpessoa/projects/loki/dataset/img02.jpg  |  sim1: [1, 0, 3]  |  sim2: [23 22  4]
Target 1: img02.jpg    (verified: True  | name: Jennifer Aniston)  |  (found 3 matches)

Current pth: /home/rpessoa/projects/loki/dataset/img03.jpg  |  sim1: [23, 22, 4]  |  sim2: [23 22  4]
Target 2: img03.jpg    (verified: True  | name: Jennifer Aniston)  |  (found 3 matches)

Current pth: /home/rpessoa/projects/loki/dataset/img04.jpg  |  sim1: [0, 3, 1]  |  sim2: [23 22  4]
Target 3: img04.jpg    (verified: True  | name: Jennifer Aniston)  |  (found 3 matches)

Current pth: /home/rpessoa/projects/loki/dataset/img07.jpg  |  sim1: [1, 3, 0, 23]  |  sim2: [23 22  4]
Target 4: img07.jpg    (verified: True  | name: Jennifer Aniston)  |  (found 3 matches)

Current pth: /home/rpessoa

In [218]:
print(similarity_obj, '\n')
print(similarity_obj2)

{'idxs': [23, 22, 4], 'threshold': 0.68, 'distances': [0.269162, 0.42606395, 0.54748183]} 

{'idxs': array([ 4, 22, 23]), 'threshold': 0.68, 'distances': [0.54748183, 0.42606395, 0.269162]}


In [223]:
similarity_obj

{'idxs': [24, 6],
 'threshold': 0.68,
 'distances': array([0.3869403, 0.4511413], dtype=float32)}